## Question Number 4

The question:
> You were asked to convert your model to API, so that the researchers can perform the alcohol identification in real-time. How would you do this (you don’t have to make the actual API)?

I'll assume that we don't need to handle the sensor readings part directly, and the input given to our API will just be the same as how it on the datasets we get. Which means we don't need to actually talked to hardware and process the signal directly. I'll also assume that this API will be impemented in python (perhaps with flask for web server) and not other backend languages like Go or C#.

### Http REST API


I think the kind of API I'll make is just simple REST API where our program will listen to its specified port (e.g. `3306`) where perhaps we could assign route `/predict` to be able to receive POST request that carries the features we need (MIP and NP readings).

When POST request received, the programs then will parse the incoming JSONs, obtain MIP and NP readings, doing pre-processing (scaling and calculating MIP/NP), call the model to make prediction, convert back the predictions into JSONs, and finally send back the result as http response.

## Step-by-step

### 1. Convert notebooks to saved the best trained model

The current state of notebooks 2 and 3 is not yet ready for creating an API, because first we need to save our models. For scikit model it could be saved into pickle file, in case of tensorflow ANN model can be saved into `.keras` file or even `.tflite` for better performance.

After the model were saved then we can begin building an http server for our program.

### 2. Building http server

In the past I've tried a simple deployment of a tensorflow model by saving the model into `.tflite` format and later use python server using flask to listen for request. Below is a rought idea of how our program will work.

 1. Here we can specified a route (e.g. `/predict`) to listen for POST request, when a request arived a function to handle it run.
 2. The function the processed the incoming JSONs, if the format isn't right we could return an error response back
 3. If the JSONs is valid, we can do pre-processing, that is scaling the data and calculating additional features (e.g. `MIP/NP ratio`)
 4. After hat we then can call our model, it is important to load the model at the start up of our program so that it will be ready and won't add up an overhead when the request is coming.
 5. We could do post-process to the prediction results of our model if necessary.
 6. Pack our predictions into the body of http response and send the response.

 Here, the API to our model is the `/predict` route with our specified formatting of the POST request body.

### 3. Deployment
Perhaps it will be wise to package everything inside a docker container, or if our program is the only service inside the VM we might as well just put our program there. Our program is very simple, it doesn't need to cache or store anything into disk or even databases. So I think the simple deployment scheme of just puting all necessary code, install all necessary packages, and starting the program will work just fine.